In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

respondent IDs for each lender

In [2]:
# banks
USBank= 504713
Wells=451965
Bell=19581

# online lenders
Quicken=7197000003
AMEC=411941324
Guaranteed=364327855


In [3]:
def LogRegress(X, y):
#####################################
# use Logistic Regression to train and test a model

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
    # create model
    LRmodel = LogisticRegression()
    # fit model with training data
#     print("X-train",X_train.head())
#     print("y_train",y_train.head())
    LRmodel.fit(X_train, y_train)
    # validate the model using testing data
    test_score=LRmodel.score(X_test, y_test)
    params=LRmodel.get_params(deep=False)
#    print("test_score",test_score)
#    print("params=",params)
    return test_score

In [4]:
def AssignData(set, data_df):
##############################################
# set = 0 use all of data_df this is all lenders
# set = 1 select from data_df for 'respondent id' in Banks
# set = 2 select from data_df for 'respondent id' in Online
# set > 3 select from data_df for 'respondent id' = set

    #reduce the data used for set > 0 to a subset of lenders or a particular lender
    if set > 3 :
        data_df=data_df.loc[data_df['respondent id'] == set,:]
    elif set == 2:
        data_df=data_df.loc[(data_df['respondent id'] == Quicken) | (data_df['respondent id'] == AMEC) | (data_df['respondent id'] == Guaranteed), :]
    elif set == 1:
        data_df=data_df.loc[(data_df['respondent id'] == USBank) | (data_df['respondent id'] == Wells) | (data_df['respondent id'] == Bell), :]
           
        # Assign X (data) and y (target)
    X=data_df.drop(["action modified","respondent id"],axis=1)
    y=data_df["action modified"]
#    print("shape", X.shape, y.shape)

    return LogRegress(X,y)

In [5]:
def LenderTest(test_data):
#################################################
# goes through the 9 tests  with one version of the test data
# 0- All data
# 1- all banks
# 2- all online lenders
# 3 USBANK 4- Wells 5- Bell 
# 6- Quicken 7- AMEC 8- Guaranteed
# returns the results in series
############################################################
    result_list=[]
    # get result for all data
    result_list.append(AssignData(0,test_data))
    # get result for all banks
    result_list.append(AssignData(1,test_data))
    #result for all online
    result_list.append(AssignData(2,test_data))
    # result for USBank, Wells, Bell 
    result_list.append(AssignData(USBank,test_data))
    result_list.append(AssignData(Wells,test_data))
    result_list.append(AssignData(Bell,test_data))
    #result for Quicken, AMEC, Guaranteed
    result_list.append(AssignData(Quicken,test_data))
    result_list.append(AssignData(AMEC,test_data))
    result_list.append(AssignData(Guaranteed,test_data))
    return result_list

In [17]:
file="AllData2017.csv"
loan_data=pd.read_csv(file)

loan_data=loan_data[["respondent id","agency code","property type","loan purpose","loan amount","county",\
                     "applicant race 1","sex","type of purchaser", "hoepa status","lien status","loan type modified",\
                     "action modified","income cleaned","income loan ratio"]]

sexless_data=loan_data.drop(["sex"], axis=1)
raceless_data=loan_data.drop(["applicant race 1"], axis=1)
loantypeless_data=loan_data.drop(["loan type modified"], axis=1) 
incomeratioless_data=loan_data.drop(["income loan ratio"], axis=1) 
incomeless_data=loan_data.drop(["income cleaned","income loan ratio"], axis=1)
loanamountless_data=loan_data.drop(["loan amount","income loan ratio"], axis=1)
financeless_data=loan_data.drop(["income cleaned","loan amount","income loan ratio"], axis=1)

sexless_data.tail()


,respondent id,agency code,property type,loan purpose,loan amount,county,applicant race 1,type of purchaser,hoepa status,lien status,loan type modified,action modified,income cleaned,income loan ratio
48024,364327855,7,1,1,486,163,5,6,2,1,1,1,144,338
48025,364327855,7,1,1,7,3,5,9,2,2,1,1,52,13
48026,364327855,7,1,1,8,53,2,9,2,2,1,1,27,30
48027,364327855,7,1,3,88,53,5,1,2,1,1,1,28,314
48028,364327855,7,1,3,102,145,5,6,2,1,1,1,32,319


In [7]:
result_df=pd.DataFrame(index=["all","banks","online","usbank","wells","bell","quicken","amec","guaranteed"])

result_df["all_fields"]=LenderTest(loan_data)
result_df["sexless"]=LenderTest(sexless_data)
result_df["raceless"]=LenderTest(raceless_data)
result_df["loantypeless"]=LenderTest(loantypeless_data)
result_df["incomeratioless"]=LenderTest(incomeratioless_data)
result_df["incomeless"]=LenderTest(incomeless_data)
result_df["loanamountless"]=LenderTest(loanamountless_data)
result_df["financeless"]=LenderTest(financeless_data)
result_df

,all_fields,sexless,raceless,loantypeless,incomeratioless
all,0.879997,0.878831,0.880247,0.870919,0.878081
banks,0.860973,0.860852,0.857819,0.855514,0.855392
online,0.962815,0.962815,0.962815,0.962815,0.962815
usbank,0.786088,0.787958,0.786088,0.777861,0.773373
wells,0.886903,0.887170,0.884503,0.886636,0.888504
bell,0.976387,0.976387,0.977485,0.973641,0.974739
quicken,0.932252,0.932252,0.932252,0.932252,0.932252
amec,0.999253,0.999253,0.999253,0.999253,0.999253
guaranteed,0.996979,0.996979,0.996979,0.996979,0.996979


In [18]:
result_df["incomeless"]=LenderTest(incomeless_data)
result_df["loanamountless"]=LenderTest(loanamountless_data)
result_df["financeless"]=LenderTest(financeless_data)
result_df

,all_fields,sexless,raceless,loantypeless,incomeratioless,incomeless,loanamountless,financeless
all,0.879997,0.878831,0.880247,0.870919,0.878081,0.874001,0.878414,0.872585
banks,0.860973,0.860852,0.857819,0.855514,0.855392,0.846779,0.856363,0.841684
online,0.962815,0.962815,0.962815,0.962815,0.962815,0.962815,0.962815,0.962815
usbank,0.786088,0.787958,0.786088,0.777861,0.773373,0.774495,0.774121,0.772251
wells,0.886903,0.887170,0.884503,0.886636,0.888504,0.883169,0.886370,0.876500
bell,0.976387,0.976387,0.977485,0.973641,0.974739,0.976387,0.975288,0.974739
quicken,0.932252,0.932252,0.932252,0.932252,0.932252,0.932252,0.932252,0.932252
amec,0.999253,0.999253,0.999253,0.999253,0.999253,0.999253,0.999253,0.999253
guaranteed,0.996979,0.996979,0.996979,0.996979,0.996979,0.996979,0.996979,0.996979
